In [2]:
from api import FPLClient

client = FPLClient()

In [3]:
client.general.team_selection_details()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],67
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],232
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],278
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],71
